In [1]:
import requests
import pandas as pd 

In [2]:
URL = "http://164.115.33.52/bs_query"
RESOURCE_ID  = "0f34f996-8c05-4a63-be63-c7d122d48e1c"

USER_TOKEN =  "089a9646-dbcd-4cc8-8868-182235c65533"  # PUT USER TOKEN HERE
 
headers = {"Authorization": USER_TOKEN}

In [3]:
def type_convert(df,column,_type) :
    if  _type == "numeric" :
        return pd.to_numeric(df[column],errors='coerce')
    elif _type == "timestamp":
        try :
            return pd.to_datetime(df[column])
        except :
            return df[column]
    else :
        return df[column]


def df_transform(df) :
    params = {"schemas" : True, "resource_id" : RESOURCE_ID}
    r = requests.get(URL,headers=headers,params=params)
    if r.status_code == 200 :
        json_res = r.json()
        schemas = json_res['schemas']
        for i in schemas :
            field = i['field']
            df[field] = type_convert(df,field,i['type'])
    else :
        raise Exception("Opp! Something went wrong")
    return df


def get_data(params):
    r = requests.get(URL,headers=headers,params=params)
    if r.status_code == 200 :
        json_res = r.json()
        return json_res['data']
    else :
        raise Exception("Opp! Something went wrong")


def get_q_list(p_params):
    params = {"q_list" : True, "resource_id" : RESOURCE_ID}
    if "last" in p_params and p_params["last"] > 0 :
        params['p_last']  = p_params["last"]
    if "from" in p_params and p_params["from"] >= 0 : 
        params['p_from']  = p_params['from']
    if "limit" in  p_params and p_params['limit'] > 0 :
        params['p_limit'] = p_params['limit']

    r = requests.get(URL,headers=headers,params=params)
    r.raise_for_status()
    if(r.status_code == 200):
        rj = r.json()
        return rj['q_list']
    else :
        raise Exception("Opp! Something went wrong")

def get_data_from_q_list (q_list) : 
    result_list = []
    for item in  q_list :
        params = {"resource_id" : RESOURCE_ID}
        params.update(item)
        dd =  get_data(params)
        if len(dd) == 0 :
            break
        result_list.extend(dd)
    return result_list

def get_data_with_params (params)  :
    q_list = get_q_list(params)
    results = get_data_from_q_list(q_list)
    return results

def get_all_data() :
    params =  {"from" : 0}
    q_list = get_q_list(params)
    results = get_data_from_q_list(q_list)
    return results

In [31]:
params = {
    "resource_id" : RESOURCE_ID,
    # "last"  : 10, # When using the 'last' parameter, the 'from' and 'limit' parameters are discarded. 
    "from" : 0, 
    "limit" : 100000,
}

In [32]:
import cProfile, pstats, io
import psutil
from pstats import SortKey


In [33]:
def main_run():
    pr = cProfile.Profile()
    pr.enable()
    # result = get_all_data()  # Use this function to retrieve all data. 
    result = get_data_with_params(params) # Use this function to retrieve data in the range defined in 'params' above. 
    pr.disable()
    df = pd.DataFrame(result)
    s = io.StringIO()
    sortby = SortKey.CUMULATIVE
    ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
    ps.strip_dirs()
    ps.print_stats('json')
    print(s.getvalue())
    return df

df = main_run()
df

         165350 function calls (165342 primitive calls) in 19.050 seconds

   Random listing order was used
   List reduced from 490 to 1 due to restriction <'json'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.002    0.001    0.301    0.075 models.py:881(json)





,brand,buzzword,cannabinoid,cannabinoid_abbreviation,category,health_benefit,id,medical_use,strain,terpene,type,uid
0,Bloom Farms,ounce,Cannabidiolic Acid,CBN,rosin,treats viral infections,9432,analgesic,Nectarine,Terpinene,hybrid,3e555380-0f85-4ae7-a123-1faa53cc62f2
1,K.I.N.D. Concentrates,high,Cannabichromic Acid,THCv,crystalline,treats psoriasis,6005,anti-ischemic,Lemon Jack,∆ 3 Carene,indica,e9ef15f9-3ce8-4e93-963f-2886f23c2f62
2,K.I.N.D. Concentrates,weed,Cannabigerol,CBGa,flower,reduces inflammation,4611,immunostimulant,Mendo Breath,R-(+)-Pulegone,indica,18cbd48e-d1cc-48ec-b83e-406c569c30c9
3,Dixie Edibles,top shelf,Cannabidivarin,∆9THC,seeds & clones,stimulates appetite,8877,anti-emetic,Bio-Diesel,α Pinene,sativa,1f2bd094-a220-4a57-ad53-f92f1ccae74b
4,Dixie Elixirs,spliff,Cannabichromene,THC,crumble,reduces risk of artery blockage,9283,anti-epileptic,Boss Hog,α Humulene,indica,5002c937-6afb-48f7-8912-90144c6f5567
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Chong's Choice,stoned,Tetrahydrocannabinol,CBCa,vape pens,reduces inflammation,3937,anti-fungal,Nectarine,Linalool,hybrid,0aec3e2a-9d52-4fcc-9f51-86ec33e82c99
99996,Siskiyou Sungrown,weed,Cannabigerol,CBD,medical,prevents stress,1521,anti-stress,Godzilla Glue,Caryophyllene Oxide,hybrid,c8b6667e-a9a0-443b-bfcc-9ef5b197b555
99997,Leafs by Snoop,keef,Tetrahydrocannabivarin,CBDv,tinctures,kills or slows bacteria growth,2920,anti-anxiety,Rude Boi OG,Limonene,indica,2f2dafe2-d6fc-4e62-8574-31ee73e3e88e
99998,Lord Jones,high,Cannabidiolic Acid,CBN,seeds & clones,reduces inflammation,2189,anti-psoriatic,Moon Glue,∆ 3 Carene,sativa,f0e82f1a-3e60-417a-b531-d7d7725034aa
